# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

Token has not been saved to git credential helper.


In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

In [3]:
torch.cuda.is_available()

True

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [4]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [5]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [ ]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [6]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/home/xyl/projects/smol-course/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/xyl/projects/smol-course/.venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [7]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 1.6774, 'grad_norm': 2.720529556274414, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
{'loss': 1.2563, 'grad_norm': 2.6444973945617676, 'learning_rate': 4.9e-05, 'epoch': 0.04}
{'loss': 1.234, 'grad_norm': 2.394711494445801, 'learning_rate': 4.85e-05, 'epoch': 0.05}
{'loss': 1.161, 'grad_norm': 2.502837657928467, 'learning_rate': 4.8e-05, 'epoch': 0.07}
{'loss': 1.0657, 'grad_norm': 2.2998547554016113, 'learning_rate': 4.75e-05, 'epoch': 0.09}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.1589809656143188, 'eval_runtime': 3.0914, 'eval_samples_per_second': 38.494, 'eval_steps_per_second': 4.852, 'epoch': 0.09}
{'loss': 1.1566, 'grad_norm': 2.2886159420013428, 'learning_rate': 4.7e-05, 'epoch': 0.11}
{'loss': 1.1648, 'grad_norm': 2.0124359130859375, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.12}
{'loss': 1.1244, 'grad_norm': 2.1284937858581543, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.14}
{'loss': 1.086, 'grad_norm': 2.224390983581543, 'learning_rate': 4.55e-05, 'epoch': 0.16}
{'loss': 1.1116, 'grad_norm': 2.0808327198028564, 'learning_rate': 4.5e-05, 'epoch': 0.18}


  0%|          | 0/15 [00:00<?, ?it/s]

...

{'eval_loss': 1.0228363275527954, 'eval_runtime': 3.1091, 'eval_samples_per_second': 38.274, 'eval_steps_per_second': 4.824, 'epoch': 1.59}
{'loss': 0.8331, 'grad_norm': 1.7076350450515747, 'learning_rate': 4.5e-06, 'epoch': 1.61}
{'loss': 0.8059, 'grad_norm': 1.8049054145812988, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.63}
{'loss': 0.8107, 'grad_norm': 2.0719659328460693, 'learning_rate': 3.5000000000000004e-06, 'epoch': 1.65}
{'loss': 0.7932, 'grad_norm': 1.7175136804580688, 'learning_rate': 3e-06, 'epoch': 1.66}
{'loss': 0.8623, 'grad_norm': 1.9925287961959839, 'learning_rate': 2.5e-06, 'epoch': 1.68}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0216706991195679, 'eval_runtime': 3.114, 'eval_samples_per_second': 38.215, 'eval_steps_per_second': 4.817, 'epoch': 1.68}
{'loss': 0.8609, 'grad_norm': 1.8237718343734741, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.7}
{'loss': 0.7802, 'grad_norm': 1.7281391620635986, 'learning_rate': 1.5e-06, 'epoch': 1.72}
{'loss': 0.7957, 'grad_norm': 1.7798866033554077, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.73}
{'loss': 0.8345, 'grad_norm': 1.7968859672546387, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.75}
{'loss': 0.7915, 'grad_norm': 1.7644240856170654, 'learning_rate': 0.0, 'epoch': 1.77}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0214849710464478, 'eval_runtime': 3.1173, 'eval_samples_per_second': 38.175, 'eval_steps_per_second': 4.812, 'epoch': 1.77}
{'train_runtime': 436.5439, 'train_samples_per_second': 9.163, 'train_steps_per_second': 2.291, 'train_loss': 0.9605711832046508, 'epoch': 1.77}


In [ ]:
trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [13]:
# Test the fine-tuned model on the same prompt
# Load the fine-tuned model
ft_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=finetune_name
).to(device)

# Load the tokenizer for the fine-tuned model
ft_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=finetune_name)

# Set up the chat format (if required)
# ft_model, ft_tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = ft_tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = ft_tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = ft_model.generate(**inputs, max_new_tokens=100)

print("After training:")
print(ft_tokenizer.decode(outputs[0], skip_special_tokens=True))

# TODO: use the fine-tuned to model generate a response, just like with the base example.



After training:
user
Write a haiku about programming
assistant
Hello! How can I help you today? I'm going to write a haiku about programming. What's programming? It's a type of art where you create a sequence of steps to solve a problem. Think of it like a recipe. You follow the steps, and then you see the result. Can you think of any programming languages? Python, Java, or JavaScript are all popular ones. What's the most popular programming language right now? Python is very popular,


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.